In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

df_train = pd.read_excel("train.xlsx")
df_test = pd.read_excel("test.xlsx")

In [ ]:
def prepare_df(df):
    df = df.copy()

    df = df.drop(["Unnamed: 0", "№ брони"], axis=1)
    df["Дата бронирования"] = pd.to_datetime(df["Дата бронирования"])

    df = df.drop(["Дата бронирования", "Заезд", "Выезд"], axis=1)

    df = df.drop("Категория номера", axis=1)

    return df

In [ ]:
df_train = prepare_df(df_train)
df_test = prepare_df(df_test)

In [ ]:
df_train["target"] = df_train["Дата отмены"].apply(lambda x: int(pd.isna(x)))
df_train = df_train.drop(["Дата отмены", "Статус брони"], axis=1)

In [ ]:
df_train

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Ночей,Источник,Гостей,Гостиница,target
0,1,25700.0,0,Внешняя система оплаты,3,Яндекс.Путешествия,2,1,0
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2,Официальный сайт,2,1,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2,Официальный сайт,2,4,1
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),1,Bronevik.com(new),1,3,1
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2,Официальный сайт,4,1,1
...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2,Официальный сайт,2,2,1
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),3,Официальный сайт,3,4,1
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),4,Официальный сайт,2,2,1
26172,1,6300.0,0,Гарантия банковской картой,1,booking.com,2,3,0


In [ ]:
X_train = df_train.drop("target", axis=1)
y_train = df_train["target"]

X_test = df_test.copy()

In [ ]:
scores = cross_val_score(
    CatBoostClassifier(cat_features=["Способ оплаты", "Источник"], verbose=0),
    X_train,
    y_train,
    cv=StratifiedKFold(),
    n_jobs=5,
    scoring="roc_auc",
)
scores.mean()

0.8155058445452095

In [ ]:
model = CatBoostClassifier(cat_features=["Способ оплаты", "Источник"])
model.fit(X_train, y_train)

Learning rate set to 0.04153
0:	learn: 0.6634974	total: 63.5ms	remaining: 1m 3s
1:	learn: 0.6278025	total: 76.5ms	remaining: 38.2s
2:	learn: 0.5986409	total: 89.2ms	remaining: 29.6s
3:	learn: 0.5690564	total: 98.5ms	remaining: 24.5s
4:	learn: 0.5432527	total: 110ms	remaining: 21.9s
5:	learn: 0.5203209	total: 120ms	remaining: 19.8s
6:	learn: 0.5018921	total: 131ms	remaining: 18.6s
7:	learn: 0.4839633	total: 139ms	remaining: 17.3s
8:	learn: 0.4677797	total: 150ms	remaining: 16.6s
9:	learn: 0.4532915	total: 160ms	remaining: 15.9s
10:	learn: 0.4392996	total: 168ms	remaining: 15.1s
11:	learn: 0.4282294	total: 178ms	remaining: 14.6s
12:	learn: 0.4188126	total: 188ms	remaining: 14.3s
13:	learn: 0.4087800	total: 199ms	remaining: 14s
14:	learn: 0.4007292	total: 212ms	remaining: 13.9s
15:	learn: 0.3921623	total: 223ms	remaining: 13.7s
16:	learn: 0.3858758	total: 234ms	remaining: 13.5s
17:	learn: 0.3799441	total: 249ms	remaining: 13.6s
18:	learn: 0.3746389	total: 259ms	remaining: 13.4s
19:	learn:

In [ ]:
pd.DataFrame(zip(X_train, model.feature_importances_)).sort_values(1, ascending=False)

,0,1
2,Внесена предоплата,46.011560
3,Способ оплаты,24.645650
1,Стоимость,10.021865
5,Источник,6.163445
4,Ночей,5.155468
6,Гостей,3.687793
7,Гостиница,3.381049
0,Номеров,0.933170


In [ ]:
pred = model.predict_proba(X_test)[:, 1]
submissiom = pd.DataFrame(pred)
submissiom.to_csv("default_catboost.csv", index=False, header=False)